In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
from pprint import pprint

from api_keys import g_key

In [23]:
output_data_file = "/Users/jordanheise/Desktop/homework-repos/python-api-challenge/city_weather.csv"

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df.head()

,City,Lat,Lng,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country
0,Port Hawkesbury,45.62,-61.35,46.99,75,1,4.70,CA
1,Bathsheba,13.22,-59.52,84.20,70,20,26.40,BB
2,Ponta do Sol,32.67,-17.10,66.20,63,40,5.82,PT
3,Berlevåg,70.86,29.09,39.20,86,75,31.09,NO
4,Östersund,63.18,14.64,32.00,95,20,11.41,SE


In [24]:
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

gmaps.configure(api_key=g_key) 

In [25]:
lat_lon = weather_data_df[['Lat', 'Lng']] 

humid = weather_data_df['Humidity (%)']

In [26]:
#create satellite map

fig = gmaps.figure()

#add heat layer to map

heatmap = gmaps.heatmap_layer(lat_lon, weights=humid, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

weather_ideal_df = pd.DataFrame(weather_data_df, columns = ["City", "Temperature (F)", "Wind Speed (mph)", "Cloudiness (%)"])

temperature = (weather_ideal_df["Temperature (F)"] <= 80) & (weather_ideal_df["Temperature (F)"] > 70)
wind_speed = weather_ideal_df["Wind Speed (mph)"] < 10
cloudiness = weather_ideal_df["Cloudiness (%)"] == 0

weather_ideal_df[temperature & wind_speed & cloudiness]

,City,Temperature (F),Wind Speed (mph),Cloudiness (%)
65,Salalah,75.20,2.24,0
86,Mīnāb,73.40,4.70,0
147,Ghorāwal,70.50,2.91,0
214,Bela,72.46,1.57,0
215,Abu Dhabi,78.80,4.70,0
245,Kiryat Gat,73.40,5.82,0
253,Sānchi,71.44,3.20,0
510,Erzin,75.99,8.01,0
541,Uthal,74.26,3.38,0


In [32]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap

hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

In [33]:
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [34]:
hotel_name

['Maritime Inn Port Hawkesbury',
 'Atlantis Hotel',
 'Morning Star Villa Madeira',
 'Berlevåg camping & Appartement AS',
 'Leopold Bed & Breakfast',
 'Lakeview Hotel Motel',
 'Lorengau Harbourside Hotel',
 nan,
 'Hotel Buganville Express',
 nan,
 'Oryx Guesthouse Oranjemund',
 "Pension Maro'i",
 'Hotel Cabo de Hornos',
 'Les Embruns Du Baril',
 'Hotel Ushuaia',
 'Southland Organic Farmstay',
 'Lighthouse Lodge & Cottages',
 nan,
 'Royal St Andrews Hotel',
 'Nemuro Grand Hotel',
 nan,
 'HOTEL HAVELI',
 'Eysturland Lodge',
 'Misty Waves Boutique Hotel',
 'Hotel Plaza Lebu',
 'Commodore on the Park',
 nan,
 'Atlanticview Cape Town Boutique Hotel',
 'Esperance Chalet Village',
 nan,
 'Qaanaaq Hotel',
 'Edgewater Inn Restaurant & Marina',
 'Green House',
 'Hotel Concha do Mar',
 'Hewing Hotel',
 'Butiama Beach Hotel',
 'Ulvesund FYR',
 'Island Way Villa',
 'Hotel Vila Gale Touros',
 nan,
 'Pemberton Valley Lodge',
 'Tynwald Willow Bend Estate',
 'De Hoop Collection',
 'Protea Hotel by Marri

In [35]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Port Hawkesbury,CA,45.62,-61.35,Maritime Inn Port Hawkesbury
1,Bathsheba,BB,13.22,-59.52,Atlantis Hotel
2,Ponta do Sol,PT,32.67,-17.10,Morning Star Villa Madeira
3,Berlevåg,NO,70.86,29.09,Berlevåg camping & Appartement AS
4,Östersund,SE,63.18,14.64,Leopold Bed & Breakfast


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))